# Coding the Frontier



## `Node` Class

We need a data structure to store information about maze cells we visit. It should be able to store the coordinates of the cell, a pointer to its "parent" -- the cell we visited immediately before we got to this cell -- and the action -- up, down, left, or right -- that brought us to this cell. How it will be used will become clearer. For now, all you need to do is follow the specification, below.

Create a class called `PathNode`. It's `__init__` method should accept three arguments (in addition to `self`):
  - `state`: the coordinates of the cell, a tuple with two integers `(row, column)`
  - `parent`: the instance of `PathNode` we visited immediately before we got to this cell
  - `action`: the action that brought us to this cell

Make sure that you "attach" these inputs to the class instances by creating properties on `self`.

Let's make sure your class works. The code, below, should create a couple of instances of `PathNode`.

In [ ]:
nodeStart = PathNode((0, 0), None, None) # parent and action are None because this is our starting point
node1 = PathNode((0, 1), nodeStart, 'right')
node2 = PathNode((1, 1), node1, 'down')
node3 = PathNode((2, 1), node2, 'down')

Because each node stores information about its "parent", we can trace the path that brought us to it. Write some code that will give you a list of actions that will get you from `startNode` to `node3`. We know what the result should be: `["right", "down", "down"]`. Make sure that's what your code produces.

## `Frontier` Class

We're going to create two versions of a frontier, one suited for depth-first searches, another suited for breadth-first searches. We want both those versions to have the same *interface*. And there's going to be plenty of code shared between the two. In such a situation, it makes sense to create a super class. We'll call it `Frontier`. We won't use it directly, but it will be the super class from which our breadth- and depth-first frontiers will inherit. 

Creating the `Frontier` class can be my job.

In [ ]:
from abc import ABC, abstractmethod

class Frontier(ABC):
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    @abstractmethod
    def remove(self):
        """Remove and return a node from the frontier."""
        pass

There's a lot going on here.
  - `Frontier` has a super class, `ABC`. That stands for `Abstract Base Class`. `ABC` gives `Frontier` some extra functionality for defining ***abstract methods*** (more on that in a moment).
  - The `__init__` method is pretty simple. It takes no inputs and creates a single attribute, `frontier`, which stores (at first, any way) an empty list.
  - Every class that inherits from `Frontier` will have an `add`, `contains_state`, and `empty` method. We can implement them on `Frontier` because their behavior will be the same for both the breadth- and depth-first frontiers. What each does is fairly straightforward:
    - `contains_state` looks through all the nodes in the `frontier` list, returning `True` if a node with matching coordinates is found and `False` otherwise.
    - `empty` returns `True` if the frontier is empty and `False` otherwise.
    - `add` appends a node to the end of the list.
  - The `remove` method is different. It's an **abstract method**. "Abstract" here just means that any class that inherits from `Frontier` has to have a method called `remove`, but it's up to the subclass to implement it. Notice the `@abstractmethod` **decorator** just above its definition. A decorator "wraps" the method and gives it extra functionality. In this case, it will ensure that subclasses of `Frontier` implement the `remove` method.

Here's what happens if you try to instantiate `Frontier` itself: 

In [ ]:
base_frontier = Frontier()

Because of that `@abstractmethod` decorator, you also can't instantiate a subclass that lacks a `remove` method.

In [ ]:
class MissingRemove(Frontier):
    pass

frontierWithoutRemove = MissingRemove()

On the other hand, there's nothing that restricts *what* a `remove` method will do. We can satisfy the abstract method will a totally inert method:

In [ ]:
class InertRemove(Frontier):
    def remove(self):
        pass

frontierInertRemove = InertRemove()

Call `remove` all you want. Nothing will happen. But the point is, it's there.

In [ ]:
frontierInertRemove.remove()
frontierInertRemove.remove()
frontierInertRemove.remove()

## Stacks & Queues

The difference between a breadth-first and a depth-first search comes down to order: which nodes do we explore next.

The difference lines up with two fundamental data structures: **stacks** and **queues**. Both are lists. But they differ in how elements are chosen from the list:
  - **stacks** are *last-out, first-out*: when it's time to take an item from the list, take the item that was most recently added
  - **queues** are *first-in, first-out*: when it's time to take an item from the list, take the oldest item (the item at the "front" of the queue)
    

![Stacks v. Queues](../img/stack-v-queue.webp)

To implement a depth-first search, we'd use a **stack** (since depth-first searches keep "expanding" the current ("last in") path).

To implement a breadth-first search, we'd use a **queue** (since breadth-first searches keep "expanding" outward on every possible path, one step at a time).

## `BreadthFirstFrontier` and `DepthFirstFrontier`


Now that we have a base class that implements most of the functionality we need, we're ready to build out a depth- and breadth-first frontier. I've stubbed them out. Both inherit all the functionality of the base `Frontier` class. All that remains is for you to implement the appropriate `remove` method for each. Remember that `BreadthFirstFrontier` should take its nodes from the **front** of the frontier (because we're implementing a **queue**) and `DepthFirstFrontier` should take its nodes from the **back** of the frontier (because we're implementing a **stack**).

Two hints:
  - remember what attributes are available because these two classes inherit from `Frontier`
  - it would be helpful if you first check if the frontier is empty

In [ ]:
class BreadthFirstFrontier(Frontier):
    def remove(self):
        pass

In [ ]:
class DepthFirstFrontier(Frontier):
    def remove(self):
        pass

To check your work, let's create some nodes, load up each frontier, and see `remove` is ordered in each of the cases. To make it a little easier to visualize, let's draw a maze and use real coordinates.

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src')) # or the path to your source code
sys.path.insert(0, module_path)

from Maze import Maze

maze = Maze("maze3")
maze.draw()

Here, then, are the first four nodes we'd add to the frontier.

In [ ]:
start = PathNode(maze.start, None, None)
node_one = PathNode((4, 0), start, 'up')
node_two = PathNode((4, 1), node_one, 'right')
node_three = PathNode((3, 1), node_two, 'up')
node_four = PathNode((4, 2), node_two, 'right')

Let's now create an instance of `BreadthFirstFrontier` and `DepthFirstFrontier` and load up each with these four nodes.

In [ ]:
bff = BreadthFirstFrontier()
dff = DepthFirstFrontier()
nodes = [start, node_one, node_two, node_three, node_four]

# add each node in the list to both frontiers using the `add` method
for node in nodes:
    bff.add(node)
    dff.add(node)

Let's see what each frontier looks like. I'll pluck out the coordinates of each node in the list.

In [ ]:
[node.state for node in bff.frontier]

In [ ]:
[node.state for node in dff.frontier]

The same, as we'd expect.

Now let's see in what order each frontier removes nodes:

In [ ]:
while bff.frontier:
    print(bff.remove().state)

In [ ]:
while dff.frontier:
    print(dff.remove().state)

If the `bff` list is opposite the `dff` list, then all is well!